In [1]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from datetime import datetime
from ipywidgets import FileUpload, VBox, HBox, Output, Button, Label, HTML, Layout
from IPython.display import display, clear_output
import os
import io

# --- Global Variables ---
apc_data = None
subject_data = None
article_data = None  # Now holds both 2023 and 2024 data
total_saved = 0
total_potential = 0
apc_file_path = "APC.csv"  # Path for preloaded APC data
subject_file_path = "subj.csv"  # Path for preloaded Subject data

# --- File Upload Widgets ---
apc_upload = FileUpload(accept='.csv', multiple=False)
subject_upload = FileUpload(accept='.csv,.xlsx', multiple=False)
article_upload = FileUpload(accept='.csv', multiple=False)

# --- Buttons ---
analyze_button = Button(description="Analyze", button_style='success')

# --- Outputs ---
upload_output = Output()
results_output = Output()

# --- Helper Functions ---

def load_preloaded_data():
    """Loads preloaded data if available."""
    global apc_data, subject_data, article_data

    with upload_output:
        if os.path.exists(apc_file_path):
            apc_data = pd.read_csv(apc_file_path)
            print(f"Loaded preloaded APC data from {apc_file_path}.")
        else:
            print("Preloaded APC data not found. Waiting for upload.")

        if os.path.exists(subject_file_path):
            subject_data = pd.read_csv(subject_file_path)
            print(f"Loaded preloaded Journal Subject data from {subject_file_path}.")
        else:
            print("Preloaded Subject data not found. Waiting for upload.")

def handle_apc_upload(change):
    global apc_data
    with upload_output:
        clear_output()
        try:
            file_info = apc_upload.value[0]
            content = file_info['content']
            apc_data = pd.read_csv(io.BytesIO(content))
            print("APC data uploaded successfully.")
        except Exception as e:
            print(f"Error processing APC upload: {e}")

def handle_subject_upload(change):
    global subject_data
    with upload_output:
        clear_output()
        try:
            file_info = subject_upload.value[0]
            content = file_info['content']
            if file_info['name'].endswith('.xlsx'):
                subject_data = pd.read_excel(io.BytesIO(content))
            else:
                subject_data = pd.read_csv(io.BytesIO(content))
            print("Journal Subject data uploaded successfully.")
        except Exception as e:
            print(f"Error processing Subject upload: {e}")

def handle_article_upload(change):
    global article_data
    with results_output:
        clear_output()
        try:
            file_info = article_upload.value[0]
            content = file_info['content']
            article_data = pd.read_csv(io.BytesIO(content))
            print("Article Details data uploaded successfully.")
            print("Preview of Article Details:")
            display(article_data.head())
        except Exception as e:
            print(f"Error processing Article Details upload: {e}")

def analyze_data(button):
    global total_saved, total_potential
    with results_output:
        clear_output()

        if apc_data is None or subject_data is None or article_data is None:
            print("Please upload all required datasets.")
            return

        # Step 1: Calculate overall Hybrid OA Ratio
        total_hybrid_oa = article_data[article_data['Article publishing model'] == 'Hybrid open access'].shape[0]
        total_publications = article_data.shape[0]
        overall_oa_ratio = total_hybrid_oa / total_publications if total_publications > 0 else 0

        # Step 2: Prepare Monthly Open Access Ratio Data
        article_data['Author journey completion date'] = pd.to_datetime(
            article_data['Author journey completion date'], errors='coerce'
        )
        article_data['Year-Month'] = article_data['Author journey completion date'].dt.to_period('M').astype(str)

        # --- Modified Monthly Hybrid OA Ratio Calculation ---
        
        # Calculate for all years
        monthly_data_all = article_data.groupby('Year-Month').agg(
            Hybrid_OA_Count=('Article publishing model', lambda x: (x == 'Hybrid open access').sum()),
            Total_Publications=('Article publishing model', 'size')
        ).reset_index()
        monthly_data_all['Hybrid OA Ratio'] = monthly_data_all['Hybrid_OA_Count'] / monthly_data_all['Total_Publications']

        # Calculate for 2023 only
        monthly_data_2023 = article_data[article_data['Year-Month'].str.startswith('2023')].groupby('Year-Month').agg(
            Hybrid_OA_Count_2023=('Article publishing model', lambda x: (x == 'Hybrid open access').sum()),
            Total_Publications_2023=('Article publishing model', 'size')
        ).reset_index()
        monthly_data_2023['Hybrid OA Ratio 2023'] = monthly_data_2023['Hybrid_OA_Count_2023'] / monthly_data_2023['Total_Publications_2023']

        # Merge the data
        monthly_data = pd.merge(monthly_data_all, monthly_data_2023, on='Year-Month', how='outer')

        # Visualization 1: Overall Hybrid OA Ratio
        display(HTML(f"<h3>Overall Hybrid Open Access Ratio</h3>"))
        display(HTML(f"<p style='font-size:20px'><b>{overall_oa_ratio:.2%}</b></p>"))

        # Visualization 2: Monthly Hybrid OA Ratio (with 2023 line)
        display(HTML(f"<h3>Monthly Hybrid Open Access Ratio Over Time</h3>"))
        fig_monthly = go.Figure()

        # Add trace for all years
        fig_monthly.add_trace(go.Scatter(
            x=monthly_data['Year-Month'],
            y=monthly_data['Hybrid OA Ratio'],
            mode='lines+markers',
            name='All Years'
        ))

        # Add trace for 2023 only
        fig_monthly.add_trace(go.Scatter(
            x=monthly_data['Year-Month'],
            y=monthly_data['Hybrid OA Ratio 2023'],
            mode='lines+markers',
            name='2023 Only'
        ))

        fig_monthly.update_layout(
            title="Monthly Hybrid Open Access Ratio Over Time",
            xaxis_title="Year-Month",
            yaxis_title="Hybrid OA Ratio",
            legend_title="Data"
        )

        fig_monthly.show()
        
        # --- End of Modified Section ---

        # --- New Visualization: Absolute Number of Publications by Type per Institution (Stacked) ---
        display(HTML(f"<h3>Absolute Number of Publications by Type per Institution (Stacked)</h3>"))

        # Group by institution and publishing model, then count publications
        institution_pub_counts = article_data.groupby(['Article Payer Institute Name', 'Article publishing model']).size().reset_index(name='Count')

        # Create the stacked bar chart
        fig_pub_counts = px.bar(
            institution_pub_counts,
            x='Article Payer Institute Name',
            y='Count',
            color='Article publishing model',
            title='Absolute Number of Publications by Type per Institution (Stacked)',
            labels={'Count': 'Number of Publications', 'Article Payer Institute Name': 'Institution', 'Article publishing model': 'Publishing Model'},
            barmode='stack'  # Use 'stack' to show bars for each type stacked on top of each other
        )

        fig_pub_counts.show()
        # --- End of New Visualization ---

        # Visualization 3: Publishing Model Ratios by Institute
        model_counts = article_data.groupby(['Article Payer Institute Name', 'Article publishing model']).size().unstack(fill_value=0)
        model_counts['Hybrid Open Access Ratio'] = model_counts.get('Hybrid open access', 0) / model_counts.sum(axis=1)
        model_counts['Subscription Ratio'] = model_counts.get('Subscription', 0) / model_counts.sum(axis=1)
        model_counts.reset_index(inplace=True)
        model_counts.sort_values(by='Hybrid Open Access Ratio', ascending=False, inplace=True)

        display(HTML(f"<h3>Publishing Model Ratios by Article Payer Institute</h3>"))
        fig = px.bar(
            model_counts,
            x='Article Payer Institute Name',
            y=['Hybrid Open Access Ratio', 'Subscription Ratio'],
            title='Publishing Model Ratios by Article Payer Institute',
            labels={'value': 'Ratio', 'variable': 'Publishing Model'},
            barmode='stack'
        )
        fig.show()

        # Step 4: Categorize Institutions Based on Impact and Improvement Feasibility
        high_volume_threshold = model_counts['Hybrid open access'].quantile(0.75)
        moderate_ratio_lower = 0.3
        moderate_ratio_upper = 0.7

        def categorize_institution(row):
            if row['Hybrid Open Access Ratio'] > 0.7:
                if row['Hybrid open access'] >= high_volume_threshold:
                    return "High Impact, High Ratio"
                else:
                    return "Low Impact, High Ratio"
            elif moderate_ratio_lower <= row['Hybrid Open Access Ratio'] <= moderate_ratio_upper:
                if row['Hybrid open access'] >= high_volume_threshold:
                    return "High Impact, Moderate Ratio"
                else:
                    return "Moderate Impact"
            elif row['Hybrid Open Access Ratio'] < moderate_ratio_lower:
                return "High Subscription Ratio"
            else:
                return "Other"

        model_counts['Impact Category'] = model_counts.apply(categorize_institution, axis=1)

        def assign_recommendations(category):
            if category == "High Impact, High Ratio":
                return "Recognize as leaders in Hybrid OA and explore opportunities for further improvement in specific disciplines."
            elif category == "High Impact, Moderate Ratio":
                return "Encourage transitioning subscription publications to Hybrid OA while applauding impact to date."
            elif category == "Moderate Impact":
                return "Inform institutions of retrospective flipping opportunities, emphasizing possible impact."
            elif category == "High Subscription Ratio":
                return "Encourage transitioning subscription publications to Hybrid OA with targeted support."
            elif category == "Low Impact, High Ratio":
                return "Maintain light engagement unless publication volume increases."
            else:
                return "Reassess strategies based on updated performance data."

        model_counts['Recommendations'] = model_counts['Impact Category'].apply(assign_recommendations)

        institution_analysis = model_counts[[
            'Article Payer Institute Name',
            'Hybrid open access',
            'Subscription',
            'Hybrid Open Access Ratio',
            'Impact Category',
            'Recommendations'
        ]].sort_values(by='Impact Category')

        display(HTML(f"<h3>4-Quadrant Analysis of Institutions</h3>"))
        fig_scatter = px.scatter(
            model_counts,
            x='Hybrid open access',
            y='Hybrid Open Access Ratio',
            size='Hybrid open access',
            color='Impact Category',
            hover_name='Article Payer Institute Name',
            hover_data={
                'Hybrid open access': True,
                'Subscription': True,
                'Hybrid Open Access Ratio': ':.2f',
                'Impact Category': True,
            },
            title="Interactive 4-Quadrant Analysis of Institutions",
            labels={
                'Hybrid open access': 'Hybrid OA Publications (Volume)',
                'Hybrid Open Access Ratio': 'Hybrid OA Ratio'
            },
            size_max=40
        )

        fig_scatter.add_hline(y=0.7, line_dash="dash", line_color="black", annotation_text="70% Ratio Threshold")
        fig_scatter.add_hline(y=0.3, line_dash="dash", line_color="black", annotation_text="30% Ratio Threshold")
        fig_scatter.add_vline(x=high_volume_threshold, line_dash="dash", line_color="black", annotation_text="High Volume Threshold")

        fig_scatter.update_layout(
            xaxis_title="Hybrid OA Publications (Volume)",
            yaxis_title="Hybrid OA Ratio",
            legend_title="Impact Category",
            template="plotly_white"
        )

        fig_scatter.show()

        # Step 5: Calculate Savings Using APC Data
        if apc_data is not None and subject_data is not None:
            hybrid_articles = article_data[article_data['Article publishing model'] == 'Hybrid open access'].rename(columns={'Journal ISSN': 'ISSN'})
            subscription_articles = article_data[article_data['Article publishing model'] == 'Subscription'].rename(columns={'Journal ISSN': 'ISSN'})

            hybrid_merged = hybrid_articles.merge(apc_data, on='ISSN', how='inner')
            subscription_merged = subscription_articles.merge(apc_data, on='ISSN', how='inner')

            hybrid_merged['Waived APC (USD)'] = pd.to_numeric(hybrid_merged['USD'], errors='coerce')
            subscription_merged['Potential APC (USD)'] = pd.to_numeric(subscription_merged['USD'], errors='coerce')

            # Institution-level savings
            hybrid_savings = hybrid_merged.groupby('Article Payer Institute Name').agg(
                Total_Savings_USD=('Waived APC (USD)', 'sum')
            ).reset_index()

            potential_savings = subscription_merged.groupby('Article Payer Institute Name').agg(
                Potential_Savings_USD=('Potential APC (USD)', 'sum')
            ).reset_index()

            savings_summary = pd.merge(hybrid_savings, potential_savings, on='Article Payer Institute Name', how='outer').fillna(0)

            total_saved = savings_summary['Total_Savings_USD'].sum()
            total_potential = savings_summary['Potential_Savings_USD'].sum()

            display(HTML(f"<h3>Institution Savings Summary</h3>"))
            display(savings_summary)

            # Visualization: Institution-level Stacked Bar Chart
            display(HTML(f"<h3>Actual and Potential APC Savings by Institution</h3>"))
            fig_institution = go.Figure()
            fig_institution.add_trace(go.Bar(
                x=savings_summary['Article Payer Institute Name'],
                y=savings_summary['Total_Savings_USD'],
                name="Actual Savings", marker_color="green"
            ))
            fig_institution.add_trace(go.Bar(
                x=savings_summary['Article Payer Institute Name'],
                y=savings_summary['Potential_Savings_USD'],
                name="Potential Savings", marker_color="orange"
            ))
            fig_institution.update_layout(
                title="Actual and Potential APC Savings by Institution",
                barmode="stack",
                xaxis_title="Institution",
                yaxis_title="Savings (USD)",
                xaxis_tickangle=90
            )
            fig_institution.add_annotation(
                xref="paper", yref="paper", x=0.5, y=1.1,
                text=f"Total Saved: ${total_saved:,.2f} | Potential Remaining: ${total_potential:,.2f}",
                showarrow=False,
                font=dict(size=14, color="black"),
                align="center"
            )
            fig_institution.show()

            # Merge with subject data
            hybrid_merged = hybrid_merged.merge(subject_data[['ISSN', 'Top Level']], on='ISSN', how='left')
            subscription_merged = subscription_merged.merge(subject_data[['ISSN', 'Top Level']], on='ISSN', how='left')

            # Top-level savings
            top_level_summary = hybrid_merged.groupby('Top Level').agg(
                Total_Hybrid_Savings=('Waived APC (USD)', 'sum'),
                Hybrid_Count=('ISSN', 'count')
            ).reset_index()

            top_level_subscription = subscription_merged.groupby('Top Level').agg(
                Total_Subscription_Savings=('Potential APC (USD)', 'sum'),
                Subscription_Count=('ISSN', 'count')
            ).reset_index()

            top_level_summary = pd.merge(
                top_level_summary,
                top_level_subscription,
                on='Top Level',
                how='outer'
            ).fillna(0)

            top_level_summary['Hybrid OA Ratio'] = top_level_summary['Hybrid_Count'] / (
                top_level_summary['Hybrid_Count'] + top_level_summary['Subscription_Count']
            )
            top_level_summary['Total Savings'] = top_level_summary['Total_Hybrid_Savings'] + top_level_summary['Total_Subscription_Savings']
            display(HTML(f"<h3>Top Level Metrics</h3>"))
            display(top_level_summary)

            # Visualization: Top Level Stacked Bar Chart
            display(HTML(f"<h3>Total Savings by Top Level</h3>"))
            fig_top_level = px.bar(
                top_level_summary,
                x='Top Level',
                y=['Total_Hybrid_Savings', 'Total_Subscription_Savings'],
                title="Total Savings by Top Level",
                labels={'value': 'Savings (USD)', 'Top Level': 'Top Level Category'},
                barmode='stack',
                template="plotly_white"
            )
            fig_top_level.show()
            
            # Calculate Hybrid OA Proportions by Institution and Top Level
            hybrid_top_level = hybrid_merged.groupby(['Article Payer Institute Name', 'Top Level']).agg(
                Hybrid_Count=('ISSN', 'count')
            ).reset_index()

            # Calculate total Hybrid OA counts for each institution
            institution_totals = hybrid_top_level.groupby('Article Payer Institute Name').agg(
                Total_Hybrid_Count=('Hybrid_Count', 'sum')
            ).reset_index()

            # Merge totals back to calculate proportions
            hybrid_top_level = hybrid_top_level.merge(
                institution_totals, on='Article Payer Institute Name', how='left'
            )
            hybrid_top_level['Proportion'] = hybrid_top_level['Hybrid_Count'] / hybrid_top_level['Total_Hybrid_Count']

            # Visualization: Stacked Bar Chart for Institutions by Top Level Proportion
            display(HTML(f"<h3>Institutional Hybrid OA Publishing Proportions by Top Level</h3>"))
            fig_institution_top_level = px.bar(
                hybrid_top_level,
                x='Article Payer Institute Name',
                y='Proportion',
                color='Top Level',
                title="Institutional Hybrid OA Publishing Proportions by Top Level",
                labels={
                    'Proportion': 'Proportion of Hybrid OA',
                    'Article Payer Institute Name': 'Institution'
                },
                template="plotly_white"
            )

            fig_institution_top_level.update_layout(
                xaxis_title="Institution",
                yaxis_title="Proportion of Hybrid OA",
                barmode='stack',
                xaxis_tickangle=45
            )

            fig_institution_top_level.show()

            # Momentum Calculation
            start_of_2024 = datetime(2024, 1, 1)
            hybrid_oa_data = article_data[(article_data['Article publishing model'] == 'Hybrid open access') &
                                        (article_data['Author journey completion date'] >= start_of_2024)]
            monthly_counts = hybrid_oa_data.groupby(['Article Payer Institute Name', 'Year-Month']).size().reset_index(name='Monthly Count')

            pivot_counts = monthly_counts.pivot(index='Article Payer Institute Name', columns='Year-Month', values='Monthly Count').fillna(0)

            rolling_3_month = pivot_counts.T.rolling(window=3).sum().T
            average_since_2024 = pivot_counts.loc[:, pivot_counts.columns >= '2024-01'].mean(axis=1)
            momentum = ((rolling_3_month.iloc[:, -1] - average_since_2024) / average_since_2024 * 100).rename('Normalized Momentum')

            model_counts = model_counts.set_index('Article Payer Institute Name')
            model_counts = model_counts.join(momentum, how='left').reset_index()

            savings_summary_table = savings_summary[['Article Payer Institute Name', 'Total_Savings_USD', 'Potential_Savings_USD']]

            model_counts = model_counts.merge(
                savings_summary_table,
                on='Article Payer Institute Name',
                how='left'
            ).fillna(0)

            model_counts.rename(columns={
                'Total_Savings_USD': 'Total Saved (USD)',
                'Potential_Savings_USD': 'Potential Saved (USD)'
            }, inplace=True)

            model_counts['Total Volume'] = model_counts['Hybrid open access'] + model_counts['Subscription']

            def categorize_momentum(value):
                if value > 10:
                    return "High Momentum"
                elif value > 0:
                    return "Moderate Momentum"
                else:
                    return "Low Momentum"

            model_counts['Momentum Category'] = model_counts['Normalized Momentum'].apply(categorize_momentum)

            # Visualization: Momentum Chart
            display(HTML(f"<h3>Institutional OA Momentum Visualization (Normalized)</h3>"))
            fig_momentum = px.scatter(
                model_counts,
                x='Hybrid open access',
                y='Normalized Momentum',
                size='Total Volume',
                color='Momentum Category',
                hover_name='Article Payer Institute Name',
                hover_data={
                    'Normalized Momentum': ':.2f',
                    'Hybrid open access': True,
                    'Subscription': True,
                    'Momentum Category': True,
                    'Total Volume': True
                },
                title="Institutional OA Momentum Visualization (Normalized)",
                labels={
                    'Hybrid open access': 'Hybrid OA Publications (Volume)',
                    'Normalized Momentum': 'Momentum Score (%)'
                },
                color_discrete_map={
                    "High Momentum": "green",
                    "Moderate Momentum": "blue",
                    "Low Momentum": "red"
                },
                size_max=40
            )

            fig_momentum.show()

# --- Event Handlers ---
apc_upload.observe(handle_apc_upload, names='value')
subject_upload.observe(handle_subject_upload, names='value')
article_upload.observe(handle_article_upload, names='value')
analyze_button.on_click(analyze_data)

# --- Layout ---
# Use an HBox for title and version side by side
title_box = HBox([
    Label("RPA"),
    Label("Version 1.10")
])

# Organize file upload widgets and labels in a VBox
upload_section = VBox([
    Label("Step 1: Upload APC Data"),
    apc_upload,
    Label("Step 2: Upload Journal Subject Data"),
    subject_upload,
    Label("Step 3: Upload Article Details Data"),
    article_upload,
    upload_output
])

# Create a VBox for the analyze button
analyze_section = VBox([
    analyze_button,
    results_output
])

# Combine everything into a main VBox
main_layout = VBox([
    title_box,        # Title and version at the top
    upload_section,   # Upload section
    analyze_section   # Analyze button and results output
])

# --- Initialize and Display ---
load_preloaded_data()  # Try to load preloaded data on startup
display(main_layout)
            